In [11]:
import pandas as pd
import re
import numpy as np

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## nomalice parameters
from unidecode import unidecode


In [12]:
df = pd.read_csv("DF_BOOTCAMPS_5158.csv")

In [13]:
dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN
dfc = dfc.reset_index(drop = True) #REINICIAR INDICE

In [14]:
dfc.shape

(1702, 29)

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [15]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'Iron Hack',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'MindHub ',
 'Muktek Academy',
 'Noukod',
 'Plataforma 5',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)']

In [16]:
# REGEX DE BOOTCAMPS ACTUALIZADAS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'iron.?hack',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'mind.?hub',
 'muktek|muk.?tek', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'worldtech$'] 

In [17]:
#CONVERTIR EN STRING
dfc["certifications"] =  dfc["certifications"].str.lower()

In [18]:
L = []

for i in dfc["certifications"]:    
    l = str(i).split("},") 
    k = [] 
    for formacion in l: 
        formacion = formacion.replace("[", "").replace("]", "") + "}" 
        k.append(formacion) 
    L.append(k)
     
L[0][0]

"{'certificate_authority': 'nepal engineering council', 'end': 'present', 'start': 'august 2014', 'title': 'certificate for the registered engineer'}"

In [19]:
K = [] 

regxCerAuth = "'certificate_authority': '.*?'"
regexTitle = "'title': '.*?'"

for individuo in L: 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        if ( len(re.findall("'certificate_authority': '.*?'", ins)) ) > 0:
            c["certificate_authority"] =  re.findall("'certificate_authority': '.*?'", ins)[0].replace("'certificate_authority': ", "").replace("'", "")

        if ( len(re.findall(regexTitle, ins)) ) > 0:
            c["title"] =  re.findall("'title': '.*?'", ins)[0].replace("'title': ", "").replace("'", "")

        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) 
    
    K.append(educacion_ind) 

K[0][0]

{'certificate_authority': 'nepal engineering council',
 'title': 'certificate for the registered engineer',
 'start': 'august 2014',
 'end': 'present'}

In [20]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfc["certifications"] = K

In [21]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfc.certifications[0][0]

{'certificate_authority': 'nepal engineering council',
 'title': 'certificate for the registered engineer',
 'start': 'august 2014',
 'end': 'present'}

In [22]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP EN EL CAMPO DE CERTIFICATION - AUTHORITY

c = []

for boot in range(len(boots)): 
    for i in range(len(dfc.certifications)): 
        for h in range(len(dfc.certifications[i])): 
            
            if "certificate_authority" in dfc.certifications[i][h]: 
                A = re.findall(boots[boot], dfc.certifications[i][h]["certificate_authority"]) 
                if len(A) > 0 : 

                    c.append((dfc.id[i], boots[boot], lista_boots[boot], dfc.certifications[i][h]['certificate_authority'], dfc.certifications[i][h]['title'], "authority"))

c = list(set(c))      
CA = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp', "certificate_authority", "title", "cert_field_match"])            

In [23]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP EN EL CAMPO DE CERTIFICATION - TITLE
d = []
for boot in range(len(boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfc.certifications)): #PARA CADA INDIVIDUO
        for h in range(len(dfc.certifications[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            # print(dfc.certifications[i][h]['certificate_authority'])

            if "title" in dfc.certifications[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                A = re.findall(boots[boot], dfc.certifications[i][h]["title"]) #REVISA SI ESTÁ EN UN BOOTCAMP

                if len(A) > 0 : #SI ESA REVISIÓN ES NO VACÍA
                    if "certificate_authority" in dfc.certifications[i][h]:
                        d.append((dfc.id[i], boots[boot], lista_boots[boot], dfc.certifications[i][h]['certificate_authority'], dfc.certifications[i][h]['title'], "title"))
                    else:
                        d.append((dfc.id[i], boots[boot], lista_boots[boot], np.nan, dfc.certifications[i][h]['title'], "title"))
d = list(set(d))      
CT = pd.DataFrame.from_records(d, columns=['id', 'regex_bootcamp', 'bootcamp', "certificate_authority", "title", "cert_field_match"])

In [24]:
cert_regex = pd.concat([CA, CT])
cert_regex = cert_regex.drop_duplicates(subset='id', keep="first")
cert_regex.to_excel("2_cert_regex.xlsx", index = False)

In [25]:
#CERT FUZZY PARTIAL

k = []
for boot in range(len(boots)):
    for i in range(len(dfc.certifications)): 
        for h in range(len(dfc.certifications[i])): 
            if "certificate_authority" in dfc.certifications[i][h]: 
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfc.certifications[i][h]["certificate_authority"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfc.certifications[i][h]["certificate_authority"])
                ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                # ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
                #print(ratio)
                ##dev f, minin string
                if ratio>85 and wordLen >=5:
                    k.append((dfc.id[i], boots[boot], lista_boots[boot], dfc.certifications[i][h]['certificate_authority'], dfc.certifications[i][h]['title'], "certificate_authority", ratio))
k = list(set(k))      
CFP = pd.DataFrame.from_records(k, columns=['id', 'regex_bootcamp', 'bootcamp', "certificate_authority", "title", "cert_field_match", "ratio"])
CFP.to_excel("2_cert_fp.xlsx", index = False)

In [26]:
#CERT FUZZY SET

u = []
for boot in range(len(boots)):
    for i in range(len(dfc.certifications)): 
        for h in range(len(dfc.certifications[i])): 
            if "certificate_authority" in dfc.certifications[i][h]: 
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfc.certifications[i][h]["certificate_authority"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfc.certifications[i][h]["certificate_authority"])
                # ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
                #print(ratio)
                ##dev f, minin string
                if ratio>85 and wordLen >=5:
                    u.append((dfc.id[i], boots[boot], lista_boots[boot], dfc.certifications[i][h]['certificate_authority'], dfc.certifications[i][h]['title'], "certificate_authority", ratio))
u = list(set(u))      
CFS = pd.DataFrame.from_records(u, columns=['id', 'regex_bootcamp', 'bootcamp', "certificate_authority", "title", "cert_field_match", "ratio"])
CFS.to_excel("2_cert_fs.xlsx", index = False)